<a href="https://colab.research.google.com/github/jdmartinezrs/incomesAndBillsProject/blob/main/Trabajo1ParaEntregar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importar Dataset**

In [30]:
from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
sheet_id ='1hSCgq0sKpE_O2XI55UfuVPkA6lYoySZSCE7KWhdxWNY'
sheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'

dp = pd.read_csv(sheet_url, header=5)
print(dp.head())

           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
0            NaN   1.0  VE-DT-CH-XMD284     CARRO TANQUE         5/23/2025   
1            NaN   2.0  VE-DT-FL-XMD588      DOBLETROQUE         5/23/2025   
2            NaN   3.0  VE-DT-FL-SUE755      DOBLETROQUE         5/23/2025   
3  5/23/25 17:32   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/23/2025   
4  5/23/25 17:32   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/23/2025   

  Km/ Hr Plan  Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
0     10,000                     45797.0        3.0    320,832      316,580    
1     10,000                     45190.0      610.0    209,543      199,850    
2      8,000                     45797.0        3.0    431,842       429,813   
3     10,000                     45797.0        3.0    269,735       263,671   
4     10,000                     45797.0        3.0    288,404      287,932    

   Fecha Cambio  Fecha Prox Cambio  Unnamed: 12 Km

In [13]:
print(dp.isna().sum())

Fecha                         8
ITEM                          4
CODIGO DE EQUIPO              4
Tipo de Vehiculo              4
fecha de revision             4
Km/ Hr Plan                   4
Fecha Ult Pre-Operacional     4
Dias Pend                     4
Ult Km/Hr                     4
Km/Hr Cambio                  4
Fecha Cambio                  4
Fecha Prox Cambio            14
Unnamed: 12                  15
Km/Hr Proyectada              4
Km/Hr Pend                    4
Progreso                      4
Observaciones                32
dtype: int64


1.1 Linpieza de datos

In [52]:
for col in dp.columns:
    if dp[col].dtype == 'object':
        dp[col] = dp[col].fillna('N/A')
    else:
        dp[col] = dp[col].fillna(0)

# Mostrar resultado
print(dp.head())

def eliminar_filas_na(dp):
    """
    Elimina filas que contienen 'N/A' en cualquier columna.
    """
    dp = dp[~dp.apply(lambda row: 'N/A' in row.values, axis=1)]
    return dp
dp = eliminar_filas_na(dp)

def convertir_coma_a_punto(dp, columna):
    """
    Convierte los valores de una columna reemplazando las comas por puntos
    y asegurando que sean numéricos.
    """
    dp[columna] = dp[columna].astype(str).str.replace(',', '.')
    dp[columna] = pd.to_numeric(dp[columna], errors='coerce')
    return dp

dp = convertir_coma_a_punto(dp, 'Km/Hr Pend')





           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
3  5/23/25 17:32   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/23/2025   
4  5/23/25 17:32   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/23/2025   
5  5/23/25 17:32   6.0  VE-DT-IN-TAW674      DOBLETROQUE         5/23/2025   
6  5/23/25 17:32   7.0  VE-DT-IN-TAW675      DOBLETROQUE         5/23/2025   
7  5/23/25 17:32   8.0  VE-DT-IN-TAW676      DOBLETROQUE         5/23/2025   

  Km/ Hr Plan  Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
3     10,000                     45797.0        3.0    269,735       263,671   
4     10,000                     45797.0        3.0    288,404      287,932    
5     12,000                     45427.0      373.0     86,614       76,403    
6     12,000                     45498.0      302.0    357,846      357,292    
7     12,000                     45402.0      398.0    375,905      376,257    

   Fecha Cambio  Fecha Prox Cambio  Unnamed: 12 Km

**1.1 Importar las liberias necesarias**

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

**2. Crear un conjunto de datos**

In [54]:
data = {
    'km_hr_pend': dp['Km/Hr Pend'].tolist(),
    #'dias_pend': dp['Dias Pend'].tolist(),
    'progreso2': dp['Progreso2'].tolist()
}
dx = pd.DataFrame(data)
print(dx)

    km_hr_pend   progreso2
0        3.936   39.360000
1        9.528   95.280000
2        1.789    0.000000
3       11.446    0.000000
4       12.352    0.000000
5        7.034   70.340000
6        5.675    0.000000
7        4.015   40.150000
8       12.049    0.000000
9       12.006    0.000000
10       4.605   46.050000
11       9.655    0.000000
12       7.087    0.000000
13       5.463   54.630000
14       3.481   34.810000
15       9.160   91.600000
16       2.769   55.380000
17       5.000  100.000000
18       3.499   58.316667
19       2.410   40.166667
20       6.274    0.000000
21       6.665    0.000000
22       7.545   75.450000
23      12.000  100.000000


In [55]:
def clasificar_riesgo(progreso2, km_hr_pend):
    if progreso2 > 90 or km_hr_pend < 0:
        return 'Alto'
    elif 70 <= progreso2 <= 90:
        return 'Medio'
    else:
        return 'Bajo'

dx['riesgo'] = dx.apply(lambda row: clasificar_riesgo(row['progreso2'], row['km_hr_pend']), axis=1)


In [56]:
print(dx.columns)


Index(['km_hr_pend', 'progreso2', 'riesgo'], dtype='object')


3.Crear y preparar los datos

In [59]:
x = dx[['km_hr_pend', 'progreso2']]
y = dx['riesgo']


4. Dividir los datos

In [60]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

5. Crear y entrenar el modelo

In [61]:
modelo_knn = KNeighborsClassifier(n_neighbors=3)
modelo_knn.fit(x_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [17]:
import numpy as np

In [64]:
import pandas as pd
import numpy as np

# Asegúrate de que los datos sean numéricos
dx['km_hr_pend'] = pd.to_numeric(dx['km_hr_pend'], errors='coerce')
#dx['dias_pend'] = pd.to_numeric(dx['dias_pend'], errors='coerce')
dx['progreso'] = pd.to_numeric(dx['progreso2'], errors='coerce')

# Definir condiciones de riesgo
conditions = [
    (dx['progreso'] > 90) | (dx['km_hr_pend'] < 0),
    (dx['progreso'] > 70) & (dx['progreso'] <= 90),
    (dx['progreso'] <= 70)
]

choices = ['Alto', 'Medio', 'Bajo']

# Especificar un valor por defecto que también sea string
dx['riesgo'] = np.select(conditions, choices, default='Desconocido')

# Mostrar resultado
print(dx[['km_hr_pend',  'progreso', 'riesgo']])
#print(dx[['km_hr_pend', 'dias_pend', 'progreso', 'riesgo']])


    km_hr_pend    progreso riesgo
0        3.936   39.360000   Bajo
1        9.528   95.280000   Alto
2        1.789    0.000000   Bajo
3       11.446    0.000000   Bajo
4       12.352    0.000000   Bajo
5        7.034   70.340000  Medio
6        5.675    0.000000   Bajo
7        4.015   40.150000   Bajo
8       12.049    0.000000   Bajo
9       12.006    0.000000   Bajo
10       4.605   46.050000   Bajo
11       9.655    0.000000   Bajo
12       7.087    0.000000   Bajo
13       5.463   54.630000   Bajo
14       3.481   34.810000   Bajo
15       9.160   91.600000   Alto
16       2.769   55.380000   Bajo
17       5.000  100.000000   Alto
18       3.499   58.316667   Bajo
19       2.410   40.166667   Bajo
20       6.274    0.000000   Bajo
21       6.665    0.000000   Bajo
22       7.545   75.450000  Medio
23      12.000  100.000000   Alto


**3.Crear y preparar los datos**


In [ ]:
#PENDIENTE POR HACER: MOSTRAR EL TIPO DE VEHICULO Y EL RIESGO